In [17]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [18]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("somanathkshirasagar/bankfaqs")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/somanathkshirasagar/bankfaqs/versions/1


In [19]:
bank = pd.read_csv("/root/.cache/kagglehub/datasets/somanathkshirasagar/bankfaqs/versions/1/BankFAQs.csv")
bank.head()

,Question,Answer,Class
0,What are the documents required for opening a ...,Following documents are required to open a Cur...,accounts
1,Can I transfer my Current Account from one bra...,"Yes, Current Accounts can be transferred from ...",accounts
2,My present status is NRI. What extra documents...,NRI/PIO can open the proprietorship/partnershi...,accounts
3,What are the documents required for opening a ...,Following documents are required for opening a...,accounts
4,What documents are required to change the addr...,Following documents are required to change the...,accounts


In [20]:
bank["content"] = bank.apply(lambda row: f"Question: {row['Question']}\nAnswer: {row['Answer']}", axis=1)
bank.head()

,Question,Answer,Class,content
0,What are the documents required for opening a ...,Following documents are required to open a Cur...,accounts,Question: What are the documents required for ...
1,Can I transfer my Current Account from one bra...,"Yes, Current Accounts can be transferred from ...",accounts,Question: Can I transfer my Current Account fr...
2,My present status is NRI. What extra documents...,NRI/PIO can open the proprietorship/partnershi...,accounts,Question: My present status is NRI. What extra...
3,What are the documents required for opening a ...,Following documents are required for opening a...,accounts,Question: What are the documents required for ...
4,What documents are required to change the addr...,Following documents are required to change the...,accounts,Question: What documents are required to chang...


In [21]:
!pip install langchain

In [22]:
from langchain.docstore.document import Document

# Prepare documents for LangChain
documents = []
for _, row in bank.iterrows():
    documents.append(Document(page_content=row["content"], metadata={"class": row["Class"]}))

In [23]:
documents[1]

Document(metadata={'class': 'accounts'}, page_content='Question: Can I transfer my Current Account from one branch to another\nAnswer: Yes, Current Accounts can be transferred from one branch to another. However, there are certain restrictions. Please visit your nearest branch for details.')

In [24]:
# !pip install langchain_community sentence-transformers

Loading into Fiass DB

In [25]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

In [26]:
!pip install faiss-cpu

In [27]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [28]:
# Generate embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS index
faiss_index = FAISS.from_documents(documents, embeddings)
faiss_index.save_local('docs/faiss_rag/')  # This will create index.faiss & index.pkl


print("FAISS vector store created and saved successfully.")


FAISS vector store created and saved successfully.


In [29]:
!pip install langchain_experimental

In [30]:
!pip install langchain-groq

In [33]:
from google.colab import userdata
# LLM with function call
from langchain_groq import ChatGroq
import os
groq_api_key=userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"]=groq_api_key
from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name='gemma2-9b-it',
    temperature=0.1
)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a5ac5e9d9d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a5ab7d15150>, model_name='gemma2-9b-it', temperature=0.1, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [38]:
from IPython.display import display, Markdown
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

question = "What is Chatbot and How it used in Finance Domain?"
# The ChatGroq class expects a list of ChatMessage objects, construct a HumanMessage with the question.
response = llm([HumanMessage(content=question)])

full_response =  f"Question: {question}\nAnswer: {response.content}" # response is a ChatMessage object, access its content
display(Markdown(colorize_text(full_response)))



**<font color='red'>Question:</font>** What is Chatbot and How it used in Finance Domain?


**<font color='green'>Answer:</font>** ## What is a Chatbot?

A chatbot is a computer program designed to simulate conversation with human users, typically through text or voice interactions. They use artificial intelligence (AI) and natural language processing (NLP) to understand user input, process it, and generate relevant responses.

Think of it like a virtual assistant that can answer questions, provide information, and even complete tasks, all through a conversational interface.

## How Chatbots are Used in Finance:

Chatbots are revolutionizing the finance industry by offering numerous benefits to both financial institutions and their customers. Here are some key applications:

**1. Customer Service:**

* **24/7 Availability:** Chatbots can handle customer queries round-the-clock, providing instant support and reducing wait times.
* **Frequently Asked Questions (FAQs):** They can answer common questions about account balances, transactions, fees, and policies, freeing up human agents for more complex issues.
* **Personalized Support:** Chatbots can access customer data to provide tailored responses and recommendations.

**2. Financial Advice & Planning:**

* **Basic Financial Guidance:** Chatbots can offer simple financial advice, such as budgeting tips, debt management strategies, and investment recommendations based on risk tolerance.
* **Goal Setting & Tracking:** They can help users set financial goals and track their progress towards achieving them.
* **Personalized Financial Plans:** Some advanced chatbots can create customized financial plans based on individual needs and circumstances.

**3. Account Management:**

* **Balance & Transaction Inquiries:** Customers can check their account balances, transaction history, and recent statements through chatbots.
* **Fund Transfers & Payments:** Chatbots can facilitate secure fund transfers between accounts and enable bill payments.
* **Account Updates & Changes:** Users can update their personal information, change passwords, and manage account settings through chatbot interactions.

**4. Fraud Detection & Prevention:**

* **Real-Time Monitoring:** Chatbots can monitor transactions for suspicious activity and alert users or authorities in case of potential fraud.
* **Security Verification:** They can use multi-factor authentication to verify user identity and prevent unauthorized access to accounts.

**5. Marketing & Sales:**

* **Lead Generation:** Chatbots can engage with potential customers, qualify leads, and collect contact information.
* **Product & Service Recommendations:** They can suggest financial products and services based on user needs and preferences.
* **Promotional Offers & Discounts:** Chatbots can deliver personalized offers and discounts to attract and retain customers.


## Benefits of Using Chatbots in Finance:

* **Improved Customer Experience:** 24/7 availability, instant responses, and personalized support enhance customer satisfaction.
* **Reduced Costs:** Chatbots automate tasks, freeing up human agents for more complex issues and reducing operational expenses.
* **Increased Efficiency:** Faster response times and streamlined processes improve overall efficiency.
* **Enhanced Security:** Chatbots can help prevent fraud and protect sensitive customer data.
* **Data-Driven Insights:** Chatbot interactions generate valuable data that can be used to improve products, services, and customer understanding.



Overall, chatbots are transforming the finance industry by providing innovative solutions that benefit both financial institutions and their customers.


In [40]:
# Define the prompt template
from langchain import PromptTemplate # Import PromptTemplate
template = """
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: {question}
Context: {context}
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)


retriever = faiss_index.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

def chat_with_rag():
    print("Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.")
    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit"]:
            break
        context = "Your context here"  # Provide context if necessary, otherwise leave it empty
        try:
            result = qa_chain({"context": context, "query": query})
            print(f"Chatbot: {result['result']}")
        except RuntimeError as e:
            print(f"RuntimeError encountered: {e}")

# Run the chat
chat_with_rag()

Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.
You: Can I transfer my Current Account from one branch to another
Chatbot: That's a good start! Here's a slightly improved response:

**Answer:**  Yes, you can usually transfer your current account to a different branch.  

To find out the specific process and any requirements, I recommend contacting your bank directly. They can provide you with the most accurate and up-to-date information. You can call their customer service line or visit your current branch. 


**Here's why the improvement is helpful:**

* **More specific:**  Instead of just saying "certain restrictions," it suggests contacting the bank for details.
* **Actionable:** It provides clear steps the customer can take to get the information they need. 
* **Emphasizes accuracy:**  It reminds the customer that bank policies can change, so getting information directly from the source is best. 

You: quit


In [41]:
import json

In [42]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saadmakhdoom/ecommerce-faq-chatbot-dataset")

print("Path to dataset files:", path)

100%|██████████| 4.30k/4.30k [00:00<00:00, 2.60MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/saadmakhdoom/ecommerce-faq-chatbot-dataset/versions/1


In [44]:
f = open("/root/.cache/kagglehub/datasets/saadmakhdoom/ecommerce-faq-chatbot-dataset/versions/1/Ecommerce_FAQ_Chatbot_dataset.json")
data = json.load(f)


In [45]:
questions = []
answers = []

for i in data["questions"]:
    questions += [i["question"]]
    answers += [i["answer"]]

In [46]:

questions[0], answers[0], data["questions"][0]

('How can I create an account?',
 "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.",
 {'question': 'How can I create an account?',
  'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."})

In [47]:
import pandas as pd
data = pd.DataFrame(questions, columns=['Questions'])
data['Answers'] = answers
data.head()

,Questions,Answers
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...


In [48]:
data["content"] = data.apply(lambda row: f"Question: {row['Questions']}\nAnswer: {row['Answers']}", axis=1)
data.head()

,Questions,Answers,content
0,How can I create an account?,"To create an account, click on the 'Sign Up' b...",Question: How can I create an account?\nAnswer...
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and...",Question: What payment methods do you accept?\...
2,How can I track my order?,You can track your order by logging into your ...,Question: How can I track my order?\nAnswer: Y...
3,What is your return policy?,Our return policy allows you to return product...,Question: What is your return policy?\nAnswer:...
4,Can I cancel my order?,You can cancel your order if it has not been s...,Question: Can I cancel my order?\nAnswer: You ...


In [49]:

from langchain.docstore.document import Document

# Prepare documents for LangChain
documents = []
for _, row in data.iterrows():
    documents.append(Document(page_content=row["content"]))

In [51]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS index
faiss_index = FAISS.from_documents(documents, embeddings)
faiss_index.save_local('docs/faiss_rag/json/')  # This will create index.faiss & index.pkl


print("FAISS vector store created and saved successfully.")

FAISS vector store created and saved successfully.


In [53]:
from IPython.display import display, Markdown
from langchain.schema import HumanMessage  # Import HumanMessage

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

question = "What is Chatbot and How it used in Finance Domain?"
# The ChatGroq class expects a list of ChatMessage objects, construct a HumanMessage with the question.
response = llm([HumanMessage(content=question)])  # Pass the question as a HumanMessage object within a list

full_response =  f"Question: {question}\nAnswer: {response.content}" # Access the content of the response
display(Markdown(colorize_text(full_response)))



**<font color='red'>Question:</font>** What is Chatbot and How it used in Finance Domain?


**<font color='green'>Answer:</font>** ## What is a Chatbot?

A chatbot is a computer program designed to simulate conversation with human users, typically through text or voice interactions. They use artificial intelligence (AI) and natural language processing (NLP) to understand user input, process it, and generate relevant responses.

Think of it like a virtual assistant that can answer questions, provide information, and even complete tasks, all through a conversational interface.

## How Chatbots are Used in Finance:

Chatbots are revolutionizing the finance industry by offering numerous benefits to both financial institutions and their customers. Here are some key applications:

**1. Customer Service:**

* **24/7 Availability:** Chatbots can handle customer queries round-the-clock, providing instant support and reducing wait times.
* **Frequently Asked Questions (FAQs):** They can efficiently answer common questions about account balances, transactions, fees, and product information.
* **Personalized Support:** Chatbots can access customer data to provide tailored responses and recommendations.

**2. Financial Advice & Guidance:**

* **Basic Financial Planning:** Chatbots can offer simple financial advice, such as budgeting tips, debt management strategies, and investment suggestions.
* **Goal Setting & Tracking:** They can help users set financial goals and track their progress towards achieving them.
* **Personalized Recommendations:** Based on user data and preferences, chatbots can suggest suitable financial products and services.

**3. Account Management:**

* **Balance & Transaction Inquiries:** Customers can check their account balances, transaction history, and recent statements through chatbots.
* **Fund Transfers & Payments:** Chatbots can facilitate secure fund transfers between accounts and enable bill payments.
* **Account Updates & Changes:** Users can update their personal information, change passwords, and manage account settings through chatbot interactions.

**4. Fraud Detection & Prevention:**

* **Real-Time Monitoring:** Chatbots can monitor transactions for suspicious activity and alert users or authorities in case of potential fraud.
* **Security Verification:** They can use multi-factor authentication to verify user identity and prevent unauthorized access to accounts.

**5. Marketing & Sales:**

* **Lead Generation:** Chatbots can engage with potential customers, qualify leads, and collect contact information.
* **Product & Service Information:** They can provide detailed information about financial products and services, answering customer queries and addressing concerns.
* **Personalized Offers & Promotions:** Chatbots can tailor marketing messages and promotions based on user preferences and financial needs.


## Benefits of Using Chatbots in Finance:

* **Improved Customer Experience:** 24/7 availability, instant responses, and personalized support enhance customer satisfaction.
* **Reduced Operational Costs:** Chatbots automate routine tasks, freeing up human agents to handle more complex issues.
* **Increased Efficiency & Productivity:** Faster response times and streamlined processes improve overall efficiency.
* **Enhanced Security:** Multi-factor authentication and fraud detection features strengthen security measures.
* **Data-Driven Insights:** Chatbot interactions provide valuable data that can be used to improve products, services, and customer targeting.



Chatbots are transforming the finance industry, offering a more convenient, efficient, and secure experience for both customers and financial institutions.


In [57]:
 #Define the prompt template
template = """
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: {question}
Context: {context}
Answer:
"""

PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

retriever = faiss_index.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

def chat_with_rag():
    print("Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.")
    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit"]:
            break
        context = "Your context here"  # Provide context if necessary, otherwise leave it empty
        try:
            result = qa_chain({"context": context, "query": query})
            print(f"Chatbot: {result['result']}")
        except RuntimeError as e:
            print(f"RuntimeError encountered: {e}")

# Run the chat
chat_with_rag()

Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.
You: How to close my account
Chatbot: Sorry, I don't know.  

While I can help with account creation, I don't have information about account closure procedures. You should look for a "Help" or "Contact Us" section on the website for instructions on closing your account. 

You: how to open an account
Chatbot: That's a good start!  Here's a slightly more comprehensive answer that could be helpful:

"To open an account with us, you can follow these steps:

1. **Visit our website:** Go to [insert website address here].
2. **Click on 'Sign Up':** You'll usually find this button in the top right corner of the page.
3. **Fill out the form:**  You'll need to provide some personal information, like your name, email address, and date of birth. You may also need to create a username and password.
4. **Verify your identity:** We may ask you to verify your identity through a link sent to your email or by providing addition